#### **Fundamentos de Bancos de Dados Relacionais e NoSQL**

#### **Conteúdo - Bases e Notebook da aula**

https://github.com/FIAP/Pos_Tech_DTAT/tree/main/Fase%203

#### **Importação de pacotes, bibliotecas e funções (def)**

In [1]:
#!pip uninstall boto3 -y
!pip uninstall botocore -y
# !pip install boto3==1.40.26 
# !pip install botocore==1.40.26
# # !pip install s3fs



Found existing installation: botocore 1.40.26
Uninstalling botocore-1.40.26:
  Successfully uninstalled botocore-1.40.26


In [2]:
#!pip install boto3==1.40.26 
!pip install botocore==1.40.26

  Using cached botocore-1.40.26-py3-none-any.whl.metadata (5.7 kB)
Using cached botocore-1.40.26-py3-none-any.whl (14.0 MB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.69
    Uninstalling botocore-1.34.69:
      Successfully uninstalled botocore-1.34.69


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.12.3 requires botocore<1.34.70,>=1.34.41, but you have botocore 1.40.26 which is incompatible.


In [1]:
import boto3, botocore
print("boto3:", boto3.__version__)
print("botocore:", botocore.__version__)


boto3: 1.40.26
botocore: 1.40.26


In [2]:
# Importar biblioteca completa
import boto3
import pandas as pd
import os
import plotly.express as px
import requests
import botocore
import psycopg2
import numpy as np
import duckdb
import mongomock
import fakeredis
import uuid
import json

# Importar função especifica de um módulo
from botocore.exceptions import BotoCoreError, ClientError
from sqlalchemy import create_engine, text, inspect
from dotenv import load_dotenv
from io import StringIO
from datetime import datetime
from pprint import pprint
from astrapy import DataAPIClient
# from cassandra.cluster import Cluster
# from cassandra.auth import PlainTextAuthProvider

#### **Testar conexão AWS via Python**

In [3]:
# Validar conexão
try:
    session = boto3.Session(profile_name="default")
    sts = session.client("sts")
    identity = sts.get_caller_identity()
    print("✅ Conectado à conta\n")
    print("UserId:", identity["UserId"])
    print("Account:", identity["Account"])
    print("Arn:", identity["Arn"])

except (BotoCoreError, ClientError) as e:
    print("❌ Erro ao conectar à AWS. Verifique suas credenciais e tente novamente.")
    print("Detalhes do erro:", e)


✅ Conectado à conta

UserId: AROAZI2LFHB2TERRDYNA4:user4377774=maradinidiego16@gmail.com
Account: 637423401077
Arn: arn:aws:sts::637423401077:assumed-role/voclabs/user4377774=maradinidiego16@gmail.com


##### **Conectar ao PostgreSQL via RDS + Executar Comandos SQL**

In [10]:
%pip install StringIO

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement StringIO (from versions: none)
ERROR: No matching distribution found for StringIO


In [4]:

load_dotenv("Users\dmaradini\.aws\credentials")

aws_access_key_id       = os.getenv("aws_access_key_id")
aws_secret_access_key   = os.getenv("aws_secret_access_key")
aws_session_token       = os.getenv("aws_session_token")  

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\dmaradini\AppData\Local\Temp\ipykernel_22456\2708774549.py:1: SyntaxWarning: invalid escape sequence '\d'
  load_dotenv("Users\dmaradini\.aws\credentials")


In [5]:
import pandas as pd
import boto3
import numpy as npfrp
from datetime import datetime
from io import StringIO
from botocore.exceptions import ClientError


session = boto3.Session(
    aws_access_key_id       =   aws_access_key_id
    ,aws_secret_access_key  =   aws_secret_access_key
    ,aws_session_token      =   aws_session_token
)



bucket_name = "techchallange-637423401077"
s3_prefix = "raw"



print("Conexão estabelecida.")

s3 = session.client('s3')
region = s3.meta.region_name or "us-east-1"

try:
    s3.head_bucket(Bucket=bucket_name)
    print(f"Bucket '{bucket_name}' já existe e está acessível.")
except ClientError as e:
    error_code = e.response['Error']['Code']
    if error_code in ("404", "NoSuchBucket"):
        print(f"Bucket '{bucket_name}' não existe, criando...\n")
        if region == "us-east-1":
            s3.create_bucket(Bucket=bucket_name)
            
        else:
            s3.create_bucket(
                Bucket=bucket_name,
                CreateBucketConfiguration={'LocationConstraint': region}
            )
        print(f"Bucket '{bucket_name}' criado com sucesso.\n")
    else:
        print(f"Erro ao acessar o bucket: {e}")
        raise


Conexão estabelecida.
Bucket 'techchallange-637423401077' já existe e está acessível.


In [ ]:
import pandas as pd
import boto3
from io import BytesIO, StringIO
from zipfile import ZipFile
import requests

bucket_name = "techchallange-637423401077"
s3_prefix = "raw"

s3 = boto3.client("s3")

# URLs dos arquivos do IBGE
urls = {
    "PNAD_COVID_052020": "https://ftp.ibge.gov.br/Trabalho_e_Rendimento/Pesquisa_Nacional_por_Amostra_de_Domicilios_PNAD_COVID19/Microdados/Dados/PNAD_COVID_052020.zip",
    "PNAD_COVID_062020": "https://ftp.ibge.gov.br/Trabalho_e_Rendimento/Pesquisa_Nacional_por_Amostra_de_Domicilios_PNAD_COVID19/Microdados/Dados/PNAD_COVID_062020.zip",
    "PNAD_COVID_072020": "https://ftp.ibge.gov.br/Trabalho_e_Rendimento/Pesquisa_Nacional_por_Amostra_de_Domicilios_PNAD_COVID19/Microdados/Dados/PNAD_COVID_072020.zip",
}

for tabela, url in urls.items():
    print(f"Processando {tabela}...")

    r = requests.get(url)
    r.raise_for_status()

    if "Dicionario" in tabela:
        # Salvar o XLS original no S3
        s3_key = f"{s3_prefix}/{tabela}.xls"
        s3.put_object(
            Bucket=bucket_name,
            Key=s3_key,
            Body=r.content  # conteúdo binário do XLS
        )
        print(f"Dicionário salvo no S3 em s3://{bucket_name}/{s3_key}")

    else:
        # Abrir o ZIP e extrair o CSV
        with ZipFile(BytesIO(r.content)) as z:
            csv_name = z.namelist()[0]
            with z.open(csv_name) as f:
                df = pd.read_csv(f, sep=";", encoding="latin1")

        # Converter para CSV em memória
        csv_buffer = StringIO()
        df.to_csv(csv_buffer, index=False)

        s3_key = f"{s3_prefix}/{tabela}.csv"
        s3.put_object(
            Bucket=bucket_name,
            Key=s3_key,
            Body=csv_buffer.getvalue()
        )
        print(f"Dados salvos no S3 em s3://{bucket_name}/{s3_key}")

print("Exportação concluída com sucesso.")


Processando PNAD_COVID_052020...
Dados salvos no S3 em s3://techchallange-637423401077/raw/PNAD_COVID_052020.csv
Processando PNAD_COVID_062020...
Dados salvos no S3 em s3://techchallange-637423401077/raw/PNAD_COVID_062020.csv
Processando PNAD_COVID_072020...
Dados salvos no S3 em s3://techchallange-637423401077/raw/PNAD_COVID_072020.csv
Processando Dicionario_PNAD_COVID_052020...
Dicionário salvo no S3 em s3://techchallange-637423401077/raw/Dicionario_PNAD_COVID_052020.xls
Processando Dicionario_PNAD_COVID_062020...
Dicionário salvo no S3 em s3://techchallange-637423401077/raw/Dicionario_PNAD_COVID_062020.xls
Processando Dicionario_PNAD_COVID_072020...
Dicionário salvo no S3 em s3://techchallange-637423401077/raw/Dicionario_PNAD_COVID_072020.xls
Exportação concluída com sucesso.


In [15]:
import boto3

session = boto3.Session(profile_name="default")
s3 = session.client("s3")
print("Consegui conectar no S3!")


Consegui conectar no S3!


##CAMADA SILVER

In [ ]:
import pandas as pd
import requests
from io import StringIO

# Lista de arquivos CSV no bucket público
bronze_files = [
    "PNAD_COVID_052020.csv",
    "PNAD_COVID_062020.csv",
    "PNAD_COVID_072020.csv"
]

# URL base do bucket público
base_url = "https://techchallange-637423401077.s3.us-east-1.amazonaws.com/raw/"

lista_de_dataframes = []

for file_name in bronze_files:
    file_url = f"{base_url}{file_name}"
    print(f"Lendo {file_url}")
    
    # Faz download do CSV como texto
    response = requests.get(file_url)
    response.raise_for_status()  # garante que deu certo
    
    # Lê o CSV como texto cru
    df_raw = pd.read_csv(StringIO(response.text), header=None)
    
    # Divide a única coluna pelo separador vírgula
    df_temp = df_raw[0].str.split(",", expand=True)
    
    # Define a primeira linha como header
    df_temp.columns = df_temp.iloc[0]
    df_temp = df_temp.drop(0).reset_index(drop=True)
    
    lista_de_dataframes.append(df_temp)

# Consolidar todos os dataframes
df_consolidado = pd.concat(lista_de_dataframes, ignore_index=True)


Lendo https://techchallange-637423401077.s3.us-east-1.amazonaws.com/raw/PNAD_COVID_052020.csv
Lendo https://techchallange-637423401077.s3.us-east-1.amazonaws.com/raw/PNAD_COVID_062020.csv
Lendo https://techchallange-637423401077.s3.us-east-1.amazonaws.com/raw/PNAD_COVID_072020.csv
(1114742, 145)
0   Ano  UF CAPITAL RM_RIDE V1008 V1012 V1013 V1016  Estrato        UPA  ...  \
0  2020  11      11            01     4    05     1  1110011  110015970  ...   
1  2020  11      11            01     4    05     1  1110011  110015970  ...   
2  2020  11      11            01     4    05     1  1110011  110015970  ...   
3  2020  11      11            01     4    05     1  1110011  110015970  ...   
4  2020  11      11            03     2    05     1  1110011  110015970  ...   

0 E001 E0021 E0022 E0023 E0024 F002A1 F002A2 F002A3 F002A4 F002A5  
0  NaN   NaN   NaN   NaN   NaN    NaN    NaN    NaN    NaN    NaN  
1  NaN   NaN   NaN   NaN   NaN    NaN    NaN    NaN    NaN    NaN  
2  NaN   NaN   NaN

In [ ]:
import boto3
import pandas as pd
from io import BytesIO

# Seu DataFrame
# df_consolidado

# Configurações do bucket
bucket_name = "techchallange-637423401077"
s3_key = "silver/df_consolidado.parquet"

# Conexão S3
s3_client = boto3.client("s3")  # precisa das credenciais configuradas no seu ambiente

# Salva o DataFrame em um buffer Parquet
buffer = BytesIO()
df_consolidado.to_parquet(buffer, index=False)

# Envia para o S3
buffer.seek(0)
s3_client.put_object(Bucket=bucket_name, Key=s3_key, Body=buffer.getvalue())

print(f"DataFrame salvo em s3://{bucket_name}/{s3_key}")


,Ano,UF,CAPITAL,RM_RIDE,V1008,V1012,V1013,V1016,Estrato,UPA,...,E001,E0021,E0022,E0023,E0024,F002A1,F002A2,F002A3,F002A4,F002A5
0,2020,11,11,,01,4,05,1,1110011,110015970,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020,11,11,,01,4,05,1,1110011,110015970,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020,11,11,,01,4,05,1,1110011,110015970,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020,11,11,,01,4,05,1,1110011,110015970,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020,11,11,,03,2,05,1,1110011,110015970,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
